# Imports

In [1]:
# import files
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

# Model (ignore)

In [ ]:
# --- Sample dataset (replace with your own later) ---
data = {
    "title": [
        "The Hobbit",
        "The Lord of the Rings",
        "Harry Potter and the Sorcerer's Stone",
        "Harry Potter and the Chamber of Secrets",
        "A Game of Thrones",
    ],
    "author": [
        "J.R.R. Tolkien",
        "J.R.R. Tolkien",
        "J.K. Rowling",
        "J.K. Rowling",
        "George R.R. Martin",
    ],
    "description": [
        "A hobbit goes on a journey with dwarves and a wizard.",
        "A group sets out to destroy a powerful ring.",
        "A boy discovers he is a wizard and attends a magical school.",
        "The young wizard faces a hidden chamber and a dark force.",
        "Noble families vie for the throne in a fantasy kingdom.",
    ],
}

books_df = pd.DataFrame(data)

# --- TF-IDF (turn text into vectors) ---
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(books_df["description"])

# --- Similarity scores ---
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


# --- Recommendation function ---
def recommend_books(title, top_n=3):
    if title not in books_df["title"].values:
        return ["Book not found in dataset."]

    idx = books_df[books_df["title"] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1 : top_n + 1]
    book_indices = [i[0] for i in sim_scores]
    return books_df["title"].iloc[book_indices].tolist()


# --- Test ---
print(recommend_books("The Hobbit"))

["Harry Potter and the Sorcerer's Stone", 'Harry Potter and the Chamber of Secrets', 'The Lord of the Rings']


# Data Processing

In [2]:
# read the data

books_df = pd.read_csv("books_data.csv")
reviews_df = pd.read_csv("Books_rating.csv")


In [3]:
def preprocess_reviews_df(reviews_df):
    """
    This function preprocesses the reviews dataframe

        Args:
            reviews_df (pd.DataFrame): dataframe of the original reviews

        Returns:
            reviews_df (pd.DataFrame): preprocessed dataframe
    """

    # rename columns for consistency
    reviews_df.rename(
        columns={
            "Id": "id",
            "Title": "title",
            "Price": "price",
            "User_id": "user_id",
            "profileName": "profile_name",
            "review/helpfulness": "helpfulness",
            "review/score": "score",
            "review/time": "review_date",
            "review/summary": "summary",
            "review/text": "text",
        },
        inplace=True,
    )

    # get the year of the review from the date (review/time)
    reviews_df["year"] = reviews_df["review_date"].apply(
        lambda x: datetime.utcfromtimestamp(x).year
    )

    # drop unnecessary cols
    reviews_df.drop(columns=["price"], inplace=True)

    # drop the null titles and users
    reviews_df = reviews_df.dropna(subset=["title", "user_id"])

    # Preprocess helpfulness
    # handle 0/0
    reviews_df["helpfulness"] = reviews_df["helpfulness"].replace("0/0", 0)

    # convert each helpfulness string to float
    reviews_df["helpfulness"] = reviews_df["helpfulness"].apply(
        lambda x: eval(x) if isinstance(x, str) and "/" in x else x
    )
    reviews_df["helpfulness"] = reviews_df["helpfulness"].astype(float)
    reviews_df["helpfulness"]

    return reviews_df

def preprocess_books_df(books_df):
    # drop irrelevant cols
    books_df.drop(columns=["previewLink", "infoLink", "ratingsCount", "publisher"], inplace=True)

    # rename cols
    books_df.rename(
        columns={
            "Title": "title",
            "publishedDate": "published_date",
        },
        inplace=True,
    )

    # drop null values in Title
    books_df.dropna(subset=["title"], inplace=True)

    # fix the dates, extract the year of the book
    books_df["published_date"] = books_df["published_date"].replace("1963*", 1963)
    books_df["published_date"] = (
        books_df["published_date"].astype(str).str.extract(r"(\d{4})")
    )
    books_df["published_date"] = books_df["published_date"].apply(
        lambda x: int(x) if isinstance(x, str) and x.isdigit() else x
    )

    # calculate age (recency feature) - possibly for content based filtering if combined with category for ex
    books_df["age"] = datetime.today().year - books_df["published_date"]
    
    return books_df

In [4]:
reviews_df = preprocess_reviews_df(reviews_df)
books_df = preprocess_books_df(books_df)

C:\Users\alano\AppData\Local\Temp\ipykernel_25152\2692198567.py:31: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  lambda x: datetime.utcfromtimestamp(x).year
C:\Users\alano\AppData\Local\Temp\ipykernel_25152\2692198567.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df["helpfulness"] = reviews_df["helpfulness"].replace("0/0", 0)
C:\Users\alano\AppData\Local\Temp\ipykernel_25152\2692198567.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [5]:
def calc_review_counts(books_df):
    
    # calculate review count for each book
    review_counts_dict = reviews_df.groupby('title').agg(
        count = ('id', 'count')
    ).to_dict()['count']

    # add the count column to books_df
    books_df['count'] = books_df['title'].apply(lambda x: review_counts_dict.get(x))

    books_df.sort_values(by='count', ascending = False)
    
    return books_df

books_df = calc_review_counts(books_df)

In [7]:
reviews_df.head()

,id,title,user_id,profile_name,helpfulness,score,review_date,summary,text,year
0,1882931173,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",1.000000,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,1999
1,0826414346,Dr. Seuss: American Icon,A30TK6U7DNS82R,Kevin Killian,1.000000,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,2004
2,0826414346,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,John Granger,0.909091,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t...",2004
3,0826414346,Dr. Seuss: American Icon,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",1.000000,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",2004
4,0826414346,Dr. Seuss: American Icon,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",1.000000,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,2005


In [8]:
books_df.head()

,title,description,authors,image,published_date,categories,age
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,1996.0,['Comics & Graphic Novels'],29.0
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,2005.0,['Biography & Autobiography'],20.0
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,2000.0,['Religion'],25.0
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,2005.0,['Fiction'],20.0
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,2003.0,NaN,22.0


# Non-personalized Recommendations

## Weighted Scoring

## Bayesian Scoring

In [ ]:
def bayesian_scoring(reviews_df):
    """
    Calculate the bayesian scoring for books based on reviews 

        Args:
            reviews_df (pd.DataFrame): dataframe of the book reviews 
        
        Returns:
            books_avg_ratings_ranked (pd.DataFrame): dataframe of the book rank based on bayesian score
    
    
    Note: function used from "Non_Personalized_Recommendations_Trending_Now" notebook from iLearn
    
    """

    #step 1: create array with title, average rating of each book, and # of ratings
    books_avg_ratings = reviews_df.groupby(['title']).agg(
        avg_rating=('score', 'mean'),  
        num_ratings=('score', 'count') 
    ).reset_index()
    
    #step 2: find m= Global average rating across all books
    m = reviews_df['score'].mean()
    #step 3: find C= confidence factor
    C = reviews_df['title'].value_counts().mean()
    #step 4: bayesian score
    books_avg_ratings['bayesian_score'] = (
        (C * m + books_avg_ratings['num_ratings'] * books_avg_ratings['avg_rating']) /
        (C + books_avg_ratings['num_ratings'])
    )
    #step 5: rank the books based on their bayesian score
    books_avg_ratings_ranked = books_avg_ratings.sort_values(by=['bayesian_score'], ascending=False)
    return books_avg_ratings_ranked

In [62]:
#Top 5 Books based on Bayesian Scoring Ranking 
top5 = bayesian_scoring(reviews_df).head()

In [9]:
import requests
from IPython.display import HTML

def show_top_books_with_covers(df, title_col='title', n=5):
    """
    show the top books with cover 

        Args:
            df (pd.DataFrame): dataframe of the top books 
            title_col (str, optional): column name that contain book titles. Default = 'title'.
            n (int, optional): no. of books to display. Default = 5.
        
        Returns:
            IPython.display.HTML: An HTML table with book covers and titles
    
    
    Note: function and HTML is a refined version of -> https://github.com/masao/google_books_api_wrapper
    
    """

    def get_google_books_cover(title):
        url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{title}"
        try:
            response = requests.get(url, timeout=5).json()
            return response['items'][0]['volumeInfo']['imageLinks']['thumbnail']
        except Exception:
            return "https://via.placeholder.com/60x90?text=No+Cover"

    top_books = df.head(n).copy()
    top_books['cover_url'] = top_books[title_col].apply(get_google_books_cover)

    # Build HTML table
    html_table = [
        "<table style='border-collapse: collapse; text-align: left;'>",
        "<tr><th>Cover</th><th>Title</th></tr>"
    ]
    for _, row in top_books.iterrows():
        html_table.append(
            f"<tr style='border: 1px solid #ccc;'>"
            f"<td><img src='{row['cover_url']}' width='60'></td>"
            f"<td style='padding: 8px;'>{row[title_col]}</td>"
            f"</tr>"
        )
    html_table.append("</table>")

    return HTML(''.join(html_table))


In [61]:
show_top_books_with_covers(top5)

Cover,Title
,Lilla Belle: The First Stages
,Why revival tarries
,With the Old Breed: At Peleliu and Okinawa
,"The Ferret Calendar 2005, Ferret Music"
,the lion's paw


## Wilson Scoring

In [ ]:
import numpy as np
def wilson_scoring(reviews_df):  
    """
    Calculate the wilson scoring for books based on reviews 

        Args:
            reviews_df (pd.DataFrame): dataframe of the book reviews 
        
        Returns:
            books_wilson (pd.DataFrame): dataframe of the book rank based on wilson score
    
    
    Note: function modified from "Non_Personalized_Recommendations_Trending_Now" notebook from iLearn
    
    """

    # step 1: review/score column will help in implementing the thumbs-up/down ratio 
    # score > 4 = thumbs-up
    #score < 4 = thumbs-down

    reviews_wilson = reviews_df
    reviews_wilson['thumbs'] = (reviews_wilson['score'] >= 4.0).astype(int)
    
    #step 2: create array with title, total(thumbs-up), total # of reviews, and p=ratio 

    books_wilson = reviews_wilson.groupby('title').agg(
        thumbs_up=('thumbs', 'sum'),      # total thumbs-up across all reviews for that title
        thumbs_down=('thumbs', lambda x: (x == 0).sum()),  # total thumbs down’s 
        total_reviews=('thumbs', 'count')         # number of review rows for that title
    ).reset_index()
    
    #step 3: get p

    books_wilson['p'] = books_wilson['thumbs_up'] / books_wilson['total_reviews']
    #to avoid dividing by 0 error
    books_wilson['p'] = books_wilson['p'].fillna(0)
    books_wilson=books_wilson.sort_values(by=['total_reviews'], ascending=False)
    
    #step 4: calculate wilson score 

    books_wilson['wilson_score'] = (
    (books_wilson['p'] + (z**2) / (2 * books_wilson['total_reviews']) -
     z * np.sqrt(
         (books_wilson['p'] * (1 - books_wilson['p']) / books_wilson['total_reviews']) +
         (z**2) / (4 * books_wilson['total_reviews']**2)
     )) /
    (1 + (z**2) / books_wilson['total_reviews'])
    )
    books_wilson['wilson_score'] = books_wilson['wilson_score'].fillna(0)
    #normalize
    books_wilson['wilson_score'] = books_wilson['wilson_score'].clip(lower=0)
    books_wilson = books_wilson.sort_values(by=['wilson_score'], ascending=False)
    return books_wilson
    


In [64]:
#Top 5 Books based on Bayesian Scoring Ranking 
top5 = wilson_scoring(reviews_df).head()
show_top_books_with_covers(top5)

Cover,Title
,With the Old Breed: At Peleliu and Okinawa
,Why revival tarries
,Wildwood Wisdom
,Little Britches
,Lilla Belle: The First Stages


## Apriori Algorithm

In [6]:
import pandas as pd
from scipy.sparse import csr_matrix
from itertools import permutations
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

def apriori_algorithm(reviews_df):

    """
    Create association rules using apriori algorithm

        Args:
            reviews_df (pd.DataFrame): dataframe of the book reviews 
        
        Returns:
            rules (pd.DataFrame): dataframe of the rules of book associations
    
    
    Note: This function is amde to handle large datasets, and has multiple preprocessing stages to achieve the best results for our goal. It includes:
          - Filtering to keep books with >500 reviews only
          - Usage of batching approach to not overwhelm the memory by taking 5000 entries at a time 
          - Filtering rules that has exactly 1 antecedent and >1 consequent

    
    """

    books_apr = reviews_df[['user_id', 'title', 'score']].copy()
    # Convert the dataset into a transactional format by considering users who have rated books ≥ 4.0 as “read”
    books_apr['read'] = (books_apr['score'] >= 4.0).astype(int)

    # calculate reveiew coounts
    books_apr = calc_review_counts(books_apr)

    # Filter to keep books with >500 reviews
    books_apr = books_apr[books_apr['count'] > 500]

    # factorize user and book IDs to get integer indices
    user_idx, unique_users = pd.factorize(books_apr["user_id"])
    book_idx, unique_books = pd.factorize(books_apr["title"])

    # build sparse matrix (1 where read = 1)
    user_book_sparse = csr_matrix(
        (books_apr["read"], (user_idx, book_idx)),
        shape=(len(unique_users), len(unique_books))
    )

    # Use batching approach to not overwhelm the memory

    subset_size = 5000
    num_subsets = user_book_sparse.shape[0] // subset_size + 1
    rules_list = []

    for i in range(num_subsets):
        start = i * subset_size
        #to avoid going out of bounds 
        end = min((i + 1) * subset_size, user_book_sparse.shape[0])

        sample_sparse = user_book_sparse[start:end, :]

        df_apriori = pd.DataFrame(sample_sparse.toarray(), columns=unique_books)
        df_apriori = df_apriori.astype(bool)

        frequent_itemsets = apriori(
        df_apriori,
        min_support=0.2,
        use_colnames=True
        )

        if frequent_itemsets.empty:
            continue
        
        frequent_itemsets.sort_values(by='support', ascending = False)

        rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
        rules = rules[
            rules['antecedents'].apply(lambda x: len(x) == 1) &
            rules['consequents'].apply(lambda x: len(x) > 1)
        ]
        rules_list.append(rules)
    # combine the rules into 1
    rules = pd.concat(rules_list, ignore_index=True)
    return rules



In [24]:
rules = apriori_algorithm(reviews_df)
books_associations = rules[['antecedents', 'consequents']]
books_associations = books_associations.rename(columns={'antecedents': 'title', 'consequents': 'titles'})
books_associations.head()

C:\Users\alano\AppData\Local\Temp\ipykernel_20560\2240377929.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rules = pd.concat(rules_list, ignore_index=True)


,title,titles
0,(Nineteen Eighty-four),"(1984, George Orwell 1984)"
1,(1984),"(Nineteen Eighty-four, George Orwell 1984)"
2,(George Orwell 1984),"(Nineteen Eighty-four, 1984)"
3,"(The Hobbit; Or, There and Back Again)","(The Hobbit, The Hobbitt, or there and back ag..."
4,(The Hobbit),"(The Hobbit; Or, There and Back Again, The Hob..."


In [28]:
def suggested_books(books_associations):
    """
    Prints associations as set of "If you read... You would like to read: ...." after each 5 rules

        Args:
            books_associations (pd.DataFrame): dataframe of the book associations with title as antecedent and titles as consequents 
        
        Returns:
            Nothing
    
    
    Note: This function is made for a more clean code for this project's purposes

    
    """
    for i in range(0, len(books_associations), 5):
        title = ", ".join(books_associations.iloc[i]['title'])
        titles = ", ".join(books_associations.iloc[i]['titles'])
        print(f"If you read: {title}\nYou would like to read: {titles}\n")
    



In [25]:
suggested_books(books_associations)

If you read: Nineteen Eighty-four
You would like to read: 1984, George Orwell 1984

If you read: The Hobbitt, or there and back again; illustrated by the author.
You would like to read: The Hobbit; Or, There and Back Again, The Hobbit

If you read: The Hobbitt, or there and back again; illustrated by the author.
You would like to read: The Hobbit, The Hobbit or There and Back Again

If you read: The Hobbit; Or, There and Back Again
You would like to read: The Hobbitt, or there and back again; illustrated by the author., The Hobbit or There and Back Again

If you read: The Hobbit or There and Back Again
You would like to read: The Hobbit There and Back Again, The Hobbitt, or there and back again; illustrated by the author.

If you read: The Hobbit
You would like to read: The Hobbit; Or, There and Back Again, The Hobbit or There and Back Again

If you read: The Hobbit There and Back Again
You would like to read: The Hobbit; Or, There and Back Again, The Hobbit or There and Back Again

If

## FP Growth

In [ ]:
def fpgrowth_algorithm(reviews_df):

    """
    Create association rules using fpgrowth algorithm

        Args:
            reviews_df (pd.DataFrame): dataframe of the book reviews 
        
        Returns:
            rules (pd.DataFrame): dataframe of the rules of book associations
    
    
    Note: This function is made to handle large datasets, and has multiple preprocessing stages to achieve the best results for our goal. It includes:
          - Min support used = 0.3 (30%) due to the large dataframe and repition of books names (1984, Ninteen eighty-four)
          - Filtering to keep books with >100 reviews only
          - Usage of batching approach to not overwhelm the memory by taking 5000 entries at a time 
          - Filtering rules that has exactly 1 antecedent and >1 consequent

    
    """

    books_fpgrowth = reviews_df[['user_id', 'title', 'score']].copy()
    # Convert the dataset into a transactional format by considering users who have rated books ≥ 4.0 as “read”
    books_fpgrowth['read'] = (books_fpgrowth['score'] >= 4.0).astype(int)

    # calculate reveiew coounts
    books_fpgrowth = calc_review_counts(books_fpgrowth)

    # Filter to keep books with >100 reviews
    books_fpgrowth = books_fpgrowth[books_fpgrowth['count'] > 100]

    # factorize user and book IDs to get integer indices
    user_idx, unique_users = pd.factorize(books_fpgrowth["user_id"])
    book_idx, unique_books = pd.factorize(books_fpgrowth["title"])

    # build sparse matrix (1 where read = 1)
    user_book_sparse = csr_matrix(
        (books_fpgrowth["read"], (user_idx, book_idx)),
        shape=(len(unique_users), len(unique_books))
    )

    # Use batching approach to not overwhelm the memory

    subset_size = 5000
    num_subsets = user_book_sparse.shape[0] // subset_size + 1
    rules_list = []

    for i in range(num_subsets):
        start = i * subset_size
        #to avoid going out of bounds 
        end = min((i + 1) * subset_size, user_book_sparse.shape[0])

        sample_sparse = user_book_sparse[start:end, :]

        df_fpgrowth = pd.DataFrame(sample_sparse.toarray(), columns=unique_books)
        df_fpgrowth = df_fpgrowth.astype(bool)

        frequent_itemsets = fpgrowth(
        df_fpgrowth,
        min_support=0.3,
        use_colnames=True
        )

        if frequent_itemsets.empty:
            continue
        
        frequent_itemsets.sort_values(by='support', ascending = False)

        rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
        rules = rules[
            rules['antecedents'].apply(lambda x: len(x) == 1) &
            rules['consequents'].apply(lambda x: len(x) > 1)
        ]
        rules_list.append(rules)
    # combine the rules into 1
    rules = pd.concat(rules_list, ignore_index=True)
    return rules



In [ ]:
rules = fpgrowth_algorithm(reviews_df)
books_associations = rules[['antecedents', 'consequents']]
books_associations = books_associations.rename(columns={'antecedents': 'title', 'consequents': 'titles'})

C:\Users\alano\AppData\Local\Temp\ipykernel_20560\1065231719.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rules = pd.concat(rules_list, ignore_index=True)


,title,titles
0,(The Hobbit There and Back Again),"(The Hobbit, The Hobbitt, or there and back ag..."
1,(The Hobbit),"(The Hobbit There and Back Again, The Hobbitt,..."
2,"(The Hobbitt, or there and back again; illustr...","(The Hobbit There and Back Again, The Hobbit)"
3,(The Hobbit There and Back Again),"(The Hobbit, The Hobbit or There and Back Again)"
4,(The Hobbit),"(The Hobbit There and Back Again, The Hobbit o..."


In [ ]:
suggested_books(books_associations)

If you read: The Hobbit There and Back Again
You would like to read: The Hobbit, The Hobbitt, or there and back again; illustrated by the author.

If you read: The Hobbit or There and Back Again
You would like to read: The Hobbit There and Back Again, The Hobbit

If you read: The Hobbitt, or there and back again; illustrated by the author.
You would like to read: The Hobbit, The Hobbit or There and Back Again

If you read: The Hobbit or There and Back Again
You would like to read: The Hobbit There and Back Again, The Hobbit, The Hobbitt, or there and back again; illustrated by the author.

If you read: The Hobbit; Or, There and Back Again
You would like to read: The Hobbit There and Back Again, The Hobbit

If you read: The Hobbit There and Back Again
You would like to read: The Hobbit; Or, There and Back Again, The Hobbitt, or there and back again; illustrated by the author.

If you read: The Hobbit or There and Back Again
You would like to read: The Hobbit; Or, There and Back Again, T

# Personalized Recommendations

## User-based collaborative filtering

In [6]:
import numpy as np
import pandas as pd
import math

In [7]:
reviews_df.head()

,id,title,user_id,profile_name,helpfulness,score,review_date,summary,text,year
0,1882931173,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",1.000000,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,1999
1,0826414346,Dr. Seuss: American Icon,A30TK6U7DNS82R,Kevin Killian,1.000000,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,2004
2,0826414346,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,John Granger,0.909091,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t...",2004
3,0826414346,Dr. Seuss: American Icon,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",1.000000,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",2004
4,0826414346,Dr. Seuss: American Icon,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",1.000000,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,2005


In [8]:
user_to_book = reviews_df.groupby('user_id')['id'].apply(list).to_dict()

In [9]:
book_to_user = reviews_df.groupby('id')['user_id'].apply(list).to_dict()

In [10]:
user_book = zip(reviews_df['user_id'], reviews_df['id'])
user_book_rating = zip(user_book, reviews_df['score'])
user_book_to_rating = dict(user_book_rating)

In [11]:
user_book_to_rating[('A30TK6U7DNS82R', '0826414346')]

5.0

In [12]:
target_user = 'A30TK6U7DNS82R'
target_user_books = set(user_to_book[target_user])

In [13]:
global_means = {}
for user, books in user_to_book.items():
    ratings = []
    for book in books:
        ratings.append(user_book_to_rating[(user, book)])
    
    global_means[user] = np.round(np.mean(ratings), 2)

In [14]:
global_means[target_user]

np.float64(4.32)

In [15]:
books_target = set(user_to_book[target_user])
mu_target = global_means[target_user]
min_common = 2  
similarity_scores = {}

In [23]:
for other_user in user_to_book:
    
    if other_user != target_user:
        books_other = set(user_to_book[other_user])
        common_books = books_target.intersection(books_other)
    
        if len(common_books) >= min_common:
            mu_other = global_means[other_user]

            numerator = 0
            denominator_target = 0
            denominator_other = 0
            
            for book in common_books:
                rating_target = user_book_to_rating[(target_user, book)]
                rating_other = user_book_to_rating[(other_user, book)]
                target_deviation = rating_target - mu_target
                other_deviation = rating_other - mu_other
                numerator += target_deviation * other_deviation
                denominator_target += target_deviation ** 2
                denominator_other += other_deviation ** 2

            denominator = math.sqrt(denominator_target) * math.sqrt(denominator_other)
            similarity = numerator / denominator if denominator != 0 else 0

        else:
            similarity = 0
    
        similarity_scores[other_user] = similarity

In [22]:
print(len(similarity_scores))
print(list(similarity_scores.items())[:5])

0
[]


In [21]:
books_target = set(user_to_book['A30TK6U7DNS82R'])

max_common = 0
most_common_user = None

for other_user, books in user_to_book.items():
    if other_user == 'A30TK6U7DNS82R':
        continue
    common_count = len(books_target.intersection(books))
    if common_count > max_common:
        max_common = common_count
        most_common_user = other_user

print("Most common books shared with another user:", max_common)
print("User with most overlap:", most_common_user)


Most common books shared with another user: 41
User with most overlap: ACIBQ6BQ6AWEV


## Item-based collaborative filtering